1.使用任意代理IP进行如下操作
2.使用requests模块进行豆瓣电影的个人用户登录操作
3.使用requests模块访问个人用户的电影排行榜->分类排行榜->任意分类对应的子页面
4.爬取需求3对应页面的电影详情数据
5.爬取3对应页面中滚动条向下拉动2000像素后加载出所有电影详情数据，存储到本地json文件中或者相应数据库中
【备注】电影详情数据包括：海报url、电影名称、导演、编剧、主演，类型，语言，上映日期，片长，豆瓣评分

In [33]:
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
}

proxy = {
    "http":"183.129.207.82:11427"
}
data = {
        "source": "movie",
        "redir": "https://movie.douban.com/",
        "form_email": "18256338296",
        "form_password": "qwaszxQAZ123",
        "login": "登录"
}

session = requests.session()
# -----------------------登录个人用户start---------------------------------
# 1.先爬取登录网址，1.用bs4获取图片url，通过图片url发起请求下载验证码图片2.获取验证码id
login_url = "https://accounts.douban.com/login"
response = requests.get(url=login_url,headers=headers,proxies=proxy)
page_text = response.text
# print(page_text)

soup= BeautifulSoup(page_text,"lxml")



# code_img_url = soup.find('img',{"id":"captcha_image"})
code_img_url = soup.select('img[id="captcha_image"]')

# 做一个判断，检验是否有验证码
if code_img_url:
    code_img_url = code_img_url[0]["src"]
    code_img = requests.get(url=code_img_url,headers=headers,proxies=proxy).content
    with open("./codeImg.png","wb") as fp:
        fp.write(code_img)
        print("验证码加载完成")

    img_id = soup.find('input',{"name":"captcha-id"})['value']# 解析出验证码id

    # 2.调用验证码识别接口，获取验证码数字，填充data
    codeText = getCode("./codeImg.png")
    print("验证码："+codeText)
    
#     将校验数据加入data中
    data["captcha-solution"] = codeText
    data["captcha-id"] = img_id
    
    
# 3.然后发起请求，得到session并下载页面



session.post(url=login_url,data=data,headers=headers,proxies=proxy)

# 获取用户登录界面
url = "https://www.douban.com/people/166383519/"
response = session.get(url=url,headers=headers,proxies=proxy)
page_text = response.text
with open("./userHome.html","w",encoding="utf-8") as fp:
    fp.write(page_text)
print("个人用户界面已加载")
# -----------------------登录个人用户end-----------------------------------


# 获取分类子页面
url = "https://movie.douban.com/typerank?type_name=%E5%96%9C%E5%89%A7&type=24&interval_id=100:90&action="
response = session.get(url=url,headers=headers,proxies=proxy)
page_text = response.text
with open("./movies.html","w",encoding="utf-8") as fp:
    fp.write(page_text)
print("电影分类子页面爬取成功")

# 获取电影详情电影详情数据json格式     这个方法可能失效，下一块代码将解决该问题
movie_data = {
    "type": "25",
    "interval_id":"100:90",
    "action":"", 
    "start": "0",
    "limit": "20",
}

url = "https://movie.douban.com/j/chart/top_list"
response = session.post(url=url,headers=headers,data=movie_data,proxies=proxy)
page_text = response.text
with open("./movies.json","w",encoding="utf-8") as fp:
    fp.write(page_text)
print("电影详情页面爬取成功")

验证码加载完成
uid: 68106
balance: 1817
cid: 1814783122, result: BROTHER
验证码：BROTHER
个人用户界面已加载
电影分类子页面爬取成功
电影详情页面爬取成功


In [34]:
# 爬取3对应页面中滚动条向下拉动2000像素后加载出所有电影详情数据(
# 需要phantomjs.exe
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from lxml import etree
import re
import json
#-------------------------需要phantomjs驱动才能进行以下操作   http://phantomjs.org/download.html
bro = webdriver.PhantomJS(executable_path="../phantomjs.exe")

url = "https://movie.douban.com/typerank?type_name=%E5%96%9C%E5%89%A7&type=24&interval_id=100:90&action="
bro.get(url=url)
sleep(3)# 让浏览器加载完数据再进行下面操作

# 滚动2000px，加载所有电影数据
js = "window.scrollTo(0,2000)"
bro.execute_script(js)
sleep(10)   #让浏览器加载完数据，再进行下列操作
bro.save_screenshot("./doubanMovie2.png")

# 获取页面数据
page_text = bro.page_source
# print(page_text)
# 解析页面数据
etree = etree.HTML(page_text)
movie_div = etree.xpath('//div[contains(@class,"movie-list-panel")]/div')
#获取 海报url、电影名称、导演、编剧、主演，类型，语言，上映日期，片长，豆瓣评分
movie_list = []
for movie in movie_div:
    try:
        movie_detail = {}
        post_url = movie.xpath('.//*[contains(@class,"movie-img")]/@data-original')[0]  # 解析出海报url   并没有src
        movie_name = movie.xpath('.//div[1]/div/div/span/a/text()')[0] #获取电影名
        rating = movie.xpath('.//span[@class="rating_num"]/text()')[0]  #获取评分
#         print("电影名：",movie_name)
#         print("评分:",rating)

        movie_url = movie.xpath('./div[@class="movie-content"]/a/@href')[0]  #获取子页面的url
        movie_response = requests.get(url = movie_url,headers=headers).text #  发起一个请求，获取电影详情子页面

        #  使用bs4，获取各种信息
        soup = BeautifulSoup(movie_response ,'lxml')
        movie_info = soup.find('div',{'id':'info'})  #获取电影详情信息

        movie_attrs = movie_info.find_all('span',class_='attrs')
        director = movie_attrs[0].text   #获取导演
        scriptwriter = ""    
        if len(movie_attrs) > 2:  
            scriptwriter = movie_attrs[1].text  #获取编剧
        actor = "无"
        if not actor:
            actor = movie_info.find('span',class_="actor").text
    #     print("演员：",actor)

        kind = movie_info.find_all('span',{'property':'v:genre'})  #类型
        kind = "/".join([i.text for i in kind])
        
        content = movie_info.text    # 获取语言
        language = re.findall(r'^语言.+',content,re.M)[0]
        language = language.split(":")[1].strip()
#         print(language)
#         print(movie_info.text)
        
        runtime = movie_info.find('span',{'property':'v:runtime'}).text  #片长
#         print("片长",runtime)
        initialReleaseDate = movie_info.find('span',{'property':'v:initialReleaseDate'})#上映时间
        if initialReleaseDate:
            initialReleaseDate = initialReleaseDate['content']
        else:
            initialReleaseDate = "无"
#         print("上映时间",initialReleaseDate)
        
#         持久化存储
        movie_detail["postUrl"] = post_url
        movie_detail["name"] = movie_name
        movie_detail["director"] = director
        movie_detail["scriptwriter"] = scriptwriter
        movie_detail["kind"] = kind
        movie_detail["language"] = language
        movie_detail["runtime"] = runtime
        movie_detail["initialReleaseDate"] = initialReleaseDate
        movie_detail["rating"] = rating
        movie_list.append(movie_detail)
        
    except IndexError as e:
        continue
# print(movie_list)
# 使用json存储
with open("./movie_detail.json","w",encoding="utf-8") as fp:
    json.dump(movie_list,fp,indent=4)

print("所有电影详情数据爬取完成")
bro.quit()

#----------------需要等待大概一分钟左右


C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


所有电影详情数据爬取完成


In [35]:
# 该函数调用了打码平台的相关接口，对指定的验证码图片进行识别，返回指定的数值
def getCode(codeImg):
    # 用户名  普通用户名
    username    = 'xgy123xx'

    # 密码
    password    = 'qwaszx'                            

    # 软件ＩＤ，开发者分成必要参数。登录开发者后台【我的软件】获得！
    appid       = 6213                                     

    # 软件密钥，开发者分成必要参数。登录开发者后台【我的软件】获得！
    appkey      = '40be8c87fc31c1a96d69b1a146c41d5a'    

    # 图片文件 下载好的验证码图片文件
    filename    = codeImg                      

    # 验证码类型，# 例：1004表示4位字母数字，不同类型收费不同。请准确填写，否则影响识别率。在此查询所有类型 http://www.yundama.com/price.html
    codetype    = 3000

    # 超时时间，秒
    timeout     = 20                                    

    # 检查
    if (username == 'username'):
        print('请设置好相关参数再测试')
    else:
        # 初始化
        yundama = YDMHttp(username, password, appid, appkey)

        # 登陆云打码
        uid = yundama.login();
        print('uid: %s' % uid)

        # 查询余额
        balance = yundama.balance();
        print('balance: %s' % balance)

        # 开始识别，图片路径，验证码类型ID，超时时间（秒），识别结果
        cid, result = yundama.decode(filename, codetype, timeout);
        print('cid: %s, result: %s' % (cid, result))
        
        return result

In [4]:
import http.client, mimetypes, urllib, json, time, requests

######################################################################

class YDMHttp:

    apiurl = 'http://api.yundama.com/api.php'
    username = ''
    password = ''
    appid = ''
    appkey = ''

    def __init__(self, username, password, appid, appkey):
        self.username = username  
        self.password = password
        self.appid = str(appid)
        self.appkey = appkey

    def request(self, fields, files=[]):
        response = self.post_url(self.apiurl, fields, files)
        response = json.loads(response)
        return response
    
    def balance(self):
        data = {'method': 'balance', 'username': self.username, 'password': self.password, 'appid': self.appid, 'appkey': self.appkey}
        response = self.request(data)
        if (response):
            if (response['ret'] and response['ret'] < 0):
                return response['ret']
            else:
                return response['balance']
        else:
            return -9001
    
    def login(self):
        data = {'method': 'login', 'username': self.username, 'password': self.password, 'appid': self.appid, 'appkey': self.appkey}
        response = self.request(data)
        if (response):
            if (response['ret'] and response['ret'] < 0):
                return response['ret']
            else:
                return response['uid']
        else:
            return -9001

    def upload(self, filename, codetype, timeout):
        data = {'method': 'upload', 'username': self.username, 'password': self.password, 'appid': self.appid, 'appkey': self.appkey, 'codetype': str(codetype), 'timeout': str(timeout)}
        file = {'file': filename}
        response = self.request(data, file)
        if (response):
            if (response['ret'] and response['ret'] < 0):
                return response['ret']
            else:
                return response['cid']
        else:
            return -9001

    def result(self, cid):
        data = {'method': 'result', 'username': self.username, 'password': self.password, 'appid': self.appid, 'appkey': self.appkey, 'cid': str(cid)}
        response = self.request(data)
        return response and response['text'] or ''

    def decode(self, filename, codetype, timeout):
        cid = self.upload(filename, codetype, timeout)
        if (cid > 0):
            for i in range(0, timeout):
                result = self.result(cid)
                if (result != ''):
                    return cid, result
                else:
                    time.sleep(1)
            return -3003, ''
        else:
            return cid, ''

    def report(self, cid):
        data = {'method': 'report', 'username': self.username, 'password': self.password, 'appid': self.appid, 'appkey': self.appkey, 'cid': str(cid), 'flag': '0'}
        response = self.request(data)
        if (response):
            return response['ret']
        else:
            return -9001

    def post_url(self, url, fields, files=[]):
        for key in files:
            files[key] = open(files[key], 'rb');
        res = requests.post(url, files=files, data=fields)
        return res.text